In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import io

# import seaborn as sns

In [9]:
def check_data(df, expected_columns, expected_dtypes):
    assert set(df.columns) == set(
        expected_columns), "Les colonnes ne correspondent pas aux attentes."
    for col, dtype in expected_dtypes.items():
        assert df[col].dtype == dtype, f"Le type de données pour {col} n'est pas {dtype}."


# Vérifier les colonnes et les types de données après transformation
expected_columns = ['CustomerID','Recency','Frequency','Monetary']
expected_dtypes = {
    'CustomerID': 'float64',
    'Recency': 'int64',
    'Frequency': 'int64',
    'Monetary': 'float64'
}

# Extraction des Données

In [8]:
# Chemin vers le fichier Excel
file_path = 'Online Retail.xlsx'
# Chargement du fichier Excel dans un DataFrame
with io.open(file_path, 'r', encoding='utf-8') as f:
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        # Affichez les premières lignes pour vérifier si le chargement s'est bien déroulé
        print(df.head())
        # Afficher les types de données de chaque colonne
        print(df.dtypes)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier Excel : {e}")
# Affichage des premières lignes du DataFrame
#print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerI

### Nettoyage des données
soustraction des valeurs négatives de la colonne Quantity

In [4]:
# Extraction des lignes avec des quantités positives
positive_quantities = df[df['Quantity'] >= 0]

# Suppression des colonnes inutiles
positive_quantities = positive_quantities.drop(
    columns=['StockCode', 'Description', 'Country'])

# Exportation des données dans un fichier CSV
positive_quantities.to_csv('positive_quantities.csv', index=False)

# Transformation des données
Création d'un nouveau dataframe issus de regroupement par client avec la récence, la somme des dépenses par client et la fréquence d'achat par client en fonction de la date de référence.

In [10]:
# Formatage de la colonne InvoiceDate en datetime
positive_quantities['InvoiceDate'] = pd.to_datetime(
    positive_quantities['InvoiceDate'])

# Calculer le montant total par transaction
positive_quantities['TotalPrice'] = positive_quantities['UnitPrice'] * \
    positive_quantities['Quantity']

# Calcul de la récence, fréquence et montant
reference_date = pd.to_datetime('2010-12-01')
# Calculer la récence
recency = positive_quantities.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days
}).rename(columns={'InvoiceDate': 'Recency'})
# Calculer la fréquence
frequency = positive_quantities.groupby('CustomerID').agg({
    'InvoiceDate': 'nunique'
}).rename(columns={'InvoiceDate': 'Frequency'})
# Calculer le montant
monetary = positive_quantities.groupby('CustomerID').agg({
    'TotalPrice': 'sum'
}).rename(columns={'TotalPrice': 'Monetary'})
# Combiner les trois métriques en un seul DataFrame
rfm = recency.join(frequency).join(monetary).reset_index()

# Verification des données
check_data(rfm, expected_columns, expected_dtypes)

# Affichage des premières lignes du DataFrame
print(rfm.head())

   CustomerID  Recency  Frequency  Monetary
0     12346.0      -49          1  77183.60
1     12347.0     -372          7   4310.00
2     12348.0     -299          4   1797.24
3     12349.0     -356          1   1757.55
4     12350.0      -64          1    334.40


### Utilisation du machine learning pour segmenter les clients (k-mean)

In [17]:
# Préparer les données pour le clustering
rfm_data = rfm[['Recency', 'Frequency', 'Monetary']]
# Standardiser les données
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_data)
# Appliquer K-means avec le nombre optimal de clusters
optimal_clusters = 5
# Initialiser K-means
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
# Prédire les clusters
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)
# Analyser les statistiques des clusters
cluster_aggregates = rfm.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
}).reset_index()

print(cluster_aggregates)

# Mapper les clusters aux noms significatifs
cluster_names = {
    # Recency moyen, Frequency moyen, Monetary moyen
    0: 'Clients Potentiellement Fidèles',
    # Recency élevé, Frequency faible, Monetary faible
    1: 'Clients Récents',
    # Recency élevé, Frequency très élevé, Monetary très élevé
    2: 'Champions',
    # Recency moyen, Frequency faible, Monetary moyen
    3: 'Clients Fidèles',
    # Recency très élevé, Frequency très élevé, Monetary très élevé
    4: 'Grands Dépenseurs'
}

# Appliquer le mapping des noms aux clusters
rfm['Cluster Name'] = rfm['Cluster'].map(cluster_names)
print(rfm.head())

# Exporter les données avec les clusters
rfm.to_csv('rfm_online_retail.csv', index=False)

# Exporter les résultats de l'analyse des clusters
cluster_aggregates['Cluster Name'] = cluster_aggregates['Cluster'].map(
    cluster_names)
cluster_aggregates.to_csv('cluster_aggregates_online_retail.csv', index=False)

   Cluster     Recency  Frequency       Monetary
0        0 -351.866606   5.242577    1997.855848
1        1 -100.262701   1.420074     510.609952
2        2 -359.459459  37.972973   25417.026486
3        3 -271.866086   2.028526     757.665160
4        4 -367.857143  64.571429  184143.690000
   CustomerID  Recency  Frequency  Monetary  Cluster  \
0     12346.0      -49          1  77183.60        2   
1     12347.0     -372          7   4310.00        0   
2     12348.0     -299          4   1797.24        3   
3     12349.0     -356          1   1757.55        0   
4     12350.0      -64          1    334.40        1   

                      Cluster Name  
0                        Champions  
1  Clients Potentiellement Fidèles  
2                  Clients Fidèles  
3  Clients Potentiellement Fidèles  
4                  Clients Récents  
